<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_colab/blob/master/Extend_dataset_YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**GO extend dataset YOLOv5**

#顔の画像から目を検出して切り抜くスクリプト
・Haarcascade_eyeを使用<br>
・目が検出できないものはskipする<br>
・横幅を1/4倍、縦幅を上下に1/4倍追加して画像を切り取る（目の全幅、眉毛が含まれるように）

・切り取った画像を横幅640pxにresizeする<br>
・フォルダ内の画像を一括変換して別フォルダに保存

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import shutil
import copy
import pandas as pd
import csv
from random import randint
from time import sleep
import numpy as np
import sys


import glob
import random
from PIL import Image
%matplotlib inline

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2

plt.ion()   # interactive mode

#Google colabをマウント

In [17]:
'''
・dlibを用いて目を切り抜く
・横幅を2倍、縦幅を上に1倍追加/下に0.5倍追加した両眼の画像が含まれるように切り取る（目の全幅、眉毛が含まれるように）
'''

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
#残り時間確認
!cat /proc/uptime | awk '{printf("残り時間 : %.2f", 12-$1/60/60)}'


残り時間 : 11.23

In [19]:
#テスト画像
test_path = '/content/drive/MyDrive/Deep_learning/Face_Images/IMG_3110.JPG'

# GO_extended_datasetを colab上のフォルダに展開
zip_path = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_extended_dataset.zip'
!unzip $zip_path -d "/content"
in_path_list  = ['/content/GO_extended_dataset/Control_photo_1886mai', '/content/GO_extended_dataset/treatable']
#保存先フォルダ
out_path_list = ['/content/GO_extended_dataset/cont_for_yolo', '/content/GO_extended_dataset/grav_for_yolo']


Archive:  /content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_extended_dataset.zip
replace /content/GO_extended_dataset/Control_photo_1886mai/1000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
N


#Haarcascadeを指定

In [5]:
# カスケードファイルのパス
eye_cascade_path = '/content/drive/My Drive/Deep_learning/haarcascade_eye.xml'
# righteye_cascade_path = '/content/drive/My Drive/Deep_learning/haarcascade_righteye_2splits.xml'
# lefteye_cascade_path = '/content/drive/My Drive/Deep_learning/haarcascade_lefteye_2splits.xml'


# カスケード分類器の特徴量取得
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)
# righteye_cascade = cv2.CascadeClassifier(eye_cascade_path)
# lefteye_cascade = cv2.CascadeClassifier(eye_cascade_path)

In [6]:
#アスペクト比を維持したまま横を400pixelに縮小する
def scale_to_width(img, width):
    scale = width / img.shape[1]
    return cv2.resize(img, dsize=None, fx=scale, fy=scale)

#図を表示する
def show_image(img):
    #img = cv2.imread(out_path)
    dst = scale_to_width(img, 200)
    cv2_imshow(dst)

# def show_image_pillow(img):
#     src = cv2.cvtColor(img_resized_list[0], cv2.COLOR_BGR2RGB)
#     plt.imshow(src)

def my_round(val, digit=0):
    p = 10 ** digit
    return int((val * p * 2 + 1) // 2 / p)

def scale_to_width(img, width):
    """幅が指定した値になるように、アスペクト比を固定して、リサイズする。
    """
    h, w = img.shape[:2]
    height = round(h * (width / w))
    dst = cv2.resize(img, dsize=(width, height))

    return dst

In [7]:
def crop_bilateral(in_path, class_num, size, showImage=True):
    img_resized_list,side_list = [],[]

    img = cv2.imread(in_path) 
    img2 = img.copy()

    if showImage:
        show_image(img)

    # 画像グレースケール化
    grayscale_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    #300pix以上のもので目に見えるものを抽出
    eye_list = eye_cascade.detectMultiScale(grayscale_img, minSize=(300, 300))
    print("")
    print('image path = ',in_path)

    # 眼検出判定
    if len(eye_list) >= 1:
        print('目が' + str(len(eye_list)) +'個検出されました')
        pass
    else:
        print("no eye detected")
        pass

    print(f"eye_list: {eye_list}")

    #画像の切り抜きと保存（連番にする）
    if len(eye_list)== 2: 

        
        for (ex, ey, ew, eh) in eye_list:
            print("[ex,ey] = %d,%d [ew,eh] = %d,%d" %(ex, ey, ew, eh))
            
            try:
                cv2.rectangle(img2, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

                #切り抜き範囲が元画像をはみ出る場合は黒画像で埋める
                top = max(0, ey-int(eh/4))
                bottom = min(grayscale_img.shape[0], int(ey + eh*5/4))
                left = max(0,int(ex-int(ew/4)))
                right = min(grayscale_img.shape[1], int(ex + ew*5/4))

                #print(f"top:{top}, bottom:{bottom}, left:{left}, right:{right}")

                img_cropped = img[top: bottom,left:right]
                height, width = img_cropped.shape[:2]

                #クロップした画像を表示
                if showImage:
                    show_image(img_cropped)
            except: 
                pass

       
        ex = min(eye_list[0][0], eye_list[1][0])
        ey = min(eye_list[0][1], eye_list[1][1])
        ew = max(eye_list[0][0]+eye_list[0][2], eye_list[1][0]+eye_list[1][2]) - ex
        eh = max(eye_list[0][1]+eye_list[0][3], eye_list[1][1]+eye_list[1][3]) - ey

        print("[ex,ey] = %d,%d [ew,eh] = %d,%d" %(ex, ey, ew, eh))

        try:
            cv2.rectangle(img2, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

            #切り抜き範囲が元画像をはみ出る場合は黒画像で埋める
            top = max(0, int(ey-eh/4))
            bottom = min(grayscale_img.shape[0], int(ey+7/6*eh))
            left = max(0,int(ex-ew/10))
            right = min(grayscale_img.shape[1], int(ex + 11/10*ew))

            print(f"top:{top}, bottom:{bottom}, left:{left}, right:{right}")

            img_cropped = img[top: bottom,left:right]
            height, width = img_cropped.shape[:2]

            img_resized = scale_to_width(img_cropped, size) #1辺を指定したpixにresize 

            #クロップした画像を表示
            if showImage:
                show_image(img_resized)
            print(img2.shape)

            #縦、横に対する割合
            X = round((right+left)/2/img2.shape[1], 6)
            Y = round((top+bottom)/2/img2.shape[0], 6)
            W = round((right-left)/img.shape[1], 6)
            H = round((bottom-top)/img.shape[0], 6)
            txt = f"{class_num} {X} {Y} {W} {H}"
            return img_resized, txt

        except:
            print('crop error')


In [ ]:
img, txt = crop_bilateral(test_path, class_num=0, size=640)
print(txt)

with open("test.txt", mode='w') as f:
    f.write(txt)


# #検出された画像を確認
# src = cv2.cvtColor(img_resized_list[0], cv2.COLOR_BGR2RGB)
# plt.imshow(src)


#**Aquisition of bounding boxes**

画像とラベルを指定フォルダに保存

In [20]:
orig_folder = "/content/GO_extended_dataset/Control_photo_1886mai" 
dst_folder = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO/cont"
class_num = 0 #classをテキストに書き込む(0:cont, 1:grav)


# orig_folder = "/content/GO_extended_dataset/treatable" 
# dst_folder = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO/grav"
# class_num = 1 #classをテキストに書き込む(0:cont, 1:grav)

path_list = glob.glob(orig_folder+"/*")
path = path_list[1]
path


'/content/GO_extended_dataset/Control_photo_1886mai/9391-20200902-84-102157_59bb947c0481af6bf0254139410a26c2f43c333726c2f3e379280c9af14bba44.jpg'

In [ ]:
path_list = glob.glob(orig_folder+"/*")
#path_list = [path_list[0]] #テスト用
dst_folder = dst_folder

#処理時間の計測
start = time.time()

#もしdst_folderがあれば削除して新しく作り直す
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
os.makedirs(f"{dst_folder}/images") #image格納用
os.makedirs(f"{dst_folder}/images_cropped") #cropped_image格納用 (YOLOでは使用しない)
os.makedirs(f"{dst_folder}/labels") #label格納用


num=0
for path in path_list:
    try: #目が検出されなかった場合のエラー回避
        img, txt = crop_bilateral(path, class_num=class_num, size=640, showImage=False)  #両眼抜き出して640pxで保存（crop時にエラーが出るものは削除される）
        img2 = cv2.imread(path).copy()
        img2 = scale_to_width(img2, 640)  #切り抜く前の画像を横のサイズを640になるように縮小
        cv2.imwrite(f"{dst_folder}/images/{os.path.basename(path).split('.')[0]}.JPG", img2) #cropせずに縮小したイメージを保存
        cv2.imwrite(f"{dst_folder}/images_cropped/{os.path.basename(path).split('.')[0]}.JPG", img) #cropしたイメージを保存する場合

        with open(f"{dst_folder}/labels/{os.path.basename(path).split('.')[0]}.txt", mode='w') as f:
            f.write(txt)
        num+=1
    except:
        pass

print("")
print('Process done!!')
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")
print (f"image_num:{num}")



image path =  /content/GO_extended_dataset/Control_photo_1886mai/6636.jpg
目が2個検出されました
eye_list: [[1402  506  693  693]
 [ 283  494  689  689]]
[ex,ey] = 1402,506 [ew,eh] = 693,693
[ex,ey] = 283,494 [ew,eh] = 689,689
[ex,ey] = 283,494 [ew,eh] = 1812,705
top:317, bottom:1316, left:101, right:2276
(1536, 2304, 3)

image path =  /content/GO_extended_dataset/Control_photo_1886mai/5910.jpg
目が2個検出されました
eye_list: [[1448  709  478  478]
 [ 264  743  568  568]]
[ex,ey] = 1448,709 [ew,eh] = 478,478
[ex,ey] = 264,743 [ew,eh] = 568,568
[ex,ey] = 264,709 [ew,eh] = 1662,602
top:558, bottom:1411, left:97, right:2092
(1536, 2304, 3)

image path =  /content/GO_extended_dataset/Control_photo_1886mai/5769.JPG
目が2個検出されました
eye_list: [[ 227  371  534  534]
 [1218  315  601  601]]
[ex,ey] = 227,371 [ew,eh] = 534,534
[ex,ey] = 1218,315 [ew,eh] = 601,601
[ex,ey] = 227,315 [ew,eh] = 1592,601
top:164, bottom:1016, left:67, right:1978
(1312, 2000, 3)

image path =  /content/GO_extended_dataset/Control_photo_1886m

#**YOLO_v5 training用フォルダを作成**

datasetをtrainとvalに分ける

https://book.st-hakky.com/docs/object-detection-yolov5-tutorial/



In [ ]:
"""
-----dataset-----train-----images
              |         |--labels 
              |--valid-----images
              |         |--labels
              |--dataset.yaml

"""

In [13]:
# split dataset

import numpy as np
from sklearn.model_selection import train_test_split
import glob
import os
import shutil

dataset_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO"

def split_dataset(dataset_dir):
    img_list = glob.glob(f"{dataset_dir}/images/*")
    img_train, img_test = train_test_split(img_list, test_size=0.3, random_state=0)

    # img_train, img_testに名前が一致するtxtファイルを抜き出す
    label_train = [f"{dataset_dir}/labels/{os.path.basename(i).split('.')[0]}.txt" for i in img_train]
    label_test = [f"{dataset_dir}/labels/{os.path.basename(i).split('.')[0]}.txt" for i in img_test]

    print(f"train: {len(label_train)},test: {len(label_test)}")

    return img_train, img_test, label_train, label_test


img_train_grav, img_test_grav, label_train_grav, label_test_grav = split_dataset(f"{dataset_dir}/grav")    
img_train_cont, img_test_cont, label_train_cont, label_test_cont = split_dataset(f"{dataset_dir}/cont")    




train: 1159,test: 498
train: 1159,test: 497


In [14]:
#YOLOv5トレーニング用
#もしdst_folderがあれば削除して新しく作り直す
dst_folder = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training"

if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
os.makedirs(f"{dst_folder}/images")
os.makedirs(f"{dst_folder}/labels")

for file in img_train_grav:
    shutil.copy(file, f"{dst_folder}/images/{os.path.basename(file)}")
for file in img_train_cont:
    shutil.copy(file, f"{dst_folder}/images/{os.path.basename(file)}")
for file in img_test_grav:
    shutil.copy(file, f"{dst_folder}/images/{os.path.basename(file)}")
for file in img_test_cont:
    shutil.copy(file, f"{dst_folder}/images/{os.path.basename(file)}")
for file in label_train_grav:
    shutil.copy(file, f"{dst_folder}/labels/{os.path.basename(file)}")
for file in label_train_grav:
    shutil.copy(file, f"{dst_folder}/labels/{os.path.basename(file)}")            
for file in label_test_cont:
    shutil.copy(file, f"{dst_folder}/labels/{os.path.basename(file)}")
for file in label_test_cont:
    shutil.copy(file, f"{dst_folder}/labels/{os.path.basename(file)}") 



In [ ]:
%cd $dst_folder

/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training


In [ ]:
%%writefile dataset.yaml
# path
train: /content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/train/images
val: /content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/valid/images

# num of classes
nc: 2

#class names
names: ['cont', 'grav'] # class名を定義

Writing dataset.yaml
